In [1]:
import pandas as pd
import os
os.chdir('../../../')
print(os.getcwd())
from v2.model import Trading

/Users/rosscopeland/Desktop/personal/code/vivaldi/back_testing


In [37]:
import sys
from v2.model import Trading
from v2.strategy.indicators.smma import SMMA
from v2.strategy.indicators.stochastic_oscillator import StochasticOscillator
from v2.strategy.indicators.bollinger_bands import BollingerBands
from v2.strategy.indicators.rsi import RSI
from v2.strategy.indicators.macd import MACD
from v2.strategy.indicators.param import Param
import collections
import numpy as np

In [3]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [4]:
def load_config():
    my_config = {}
    with open('config.config') as config:
        for line in config:
            args = line.split('=')
            my_config[args[0]] = args[1].rstrip().split(',')
    return my_config

model = Trading(load_config())

In [5]:
#only one data set rn
if len(model.dfs) != 1:
    raise ValueError("only one dataset to be used here")

dataset = model.dfs[0][0]
buy_prices = []
buy_times = []
sell_prices = []
sell_times = []
name = ''
candle, buys, sells = None, None, None
fee = 0.0026


cur_min = sys.maxsize
cur_max = 0.00
cur_min_time = 00.0
cur_max_time = 00.0
last_buy_price = 000.00
last_buy_time = 000.00
sell_time = 0000.0
looking_for_buy = True
for row in dataset.itertuples():
    close = row.close
    time = row.time
    if looking_for_buy:
        if close < cur_min:
            cur_min = close
            cur_min_time = time
            cur_max = close
            cur_max_time = time
        elif close > cur_max:
            cur_max = close
            cur_max_time = time
            delta = cur_max - cur_min
            if delta > ((cur_min * fee) + (cur_max * fee)):
                looking_for_buy = False
                last_buy_price = cur_min
                last_buy_time = cur_min_time
    else:
        trade_fee = ((cur_min * fee) + (cur_max * fee))
        if close > cur_max:
            cur_max = close
            cur_max_time = time
        elif close < cur_max - trade_fee:
            looking_for_buy = True
            buy_prices.append(cur_min)
            buy_times.append(cur_min_time)
            sell_prices.append(cur_max)
            sell_times.append(cur_max_time)
            cur_min = close
            cur_min_time = time
            cur_max = close
            cur_max_time = time

In [6]:
#making set of buy and sell times
optimal_buy_times = set(sell_times)
optimal_sell_times = set(buy_times)

In [7]:
def identify_peak_trough(row):
    if row.time in optimal_buy_times and row.time in optimal_sell_times:
        raise ValueError("There's something wrong with the algo")
    elif row.time in optimal_buy_times:
        return -1
    elif row.time in optimal_sell_times:
        return 1
    else:
        return 0

In [8]:
dataset["peak_trough_wait"] = dataset.apply(lambda x: identify_peak_trough(x), axis=1)

In [9]:
print(len(optimal_buy_times), len(optimal_sell_times))

33601 33601


In [10]:
#lets add in some indicators
training_set = dataset
appended_dataset = pd.DataFrame()
ema_fast = Param(5, 10000, 0, 'ema_fast', 60)
ema_slow = Param(6, 10001, 0, 'ema_slow', 120)
signal = Param(5, 10001, 0, 'signal', 90)
macd_ = MACD(_params=[ema_fast, ema_slow, signal], _name='macd')
macd_.genData(training_set, gen_new_values=False)
boll_period = Param(5, 10000, 0, 'period', 90)
boll_bands = BollingerBands(_params=[boll_period], _name='bollinger_bands')
boll_bands.genData(training_set, gen_new_values=False)
stoch_highlow = Param(5, 10000, 0, 'highlow_range', 90.0)
stoch_k = Param(5, 10000, 0, 'k_period', 270.0)
stoch_oscillator = StochasticOscillator(_params=[stoch_highlow, stoch_k], _name='stochastic_oscillator')
stoch_oscillator.genData(training_set, gen_new_values=False)
rsi_period = Param(5, 10000, 0, 'period', 90.0)
rsi_ = RSI(_params=[rsi_period], _name='rsi')
rsi_.genData(training_set, gen_new_values=False)
smma_period = Param(5, 10000, 0, 'period', 90.0)
smma_ = SMMA(_params=[smma_period], _name='smma')
smma_.genData(training_set, gen_new_values=False)
training_set['slope'] = (training_set['close'].rolling(window=30).max() - training_set['close'].rolling(window=30).min()) / training_set['close'].rolling(window=30).max()
training_set = training_set.dropna()
appended_dataset = appended_dataset.append(training_set)

In [11]:
appended_dataset

,time,open,high,low,close,volume,trades,peak_trough_wait,ema_slow,ema_fast,...,stosc_k,stosc_d,rsi_diff,rsi_u,rsi_d,rsi_smma_u,rsi_smma_d,rsi,smma,slope
358,1383503820,207.67198,207.67198,207.67198,207.67198,0.100000,1,0,203.646855,205.570957,...,4.275846,23.660815,-0.00418,0.00000,0.00418,0.328056,0.243793,57.367632,201.724654,0.013416
359,1383505860,207.43988,207.43988,206.89000,207.01377,1.263692,5,0,203.702506,205.618262,...,13.776266,23.698312,-0.65821,0.00000,0.65821,0.324344,0.248482,56.621691,201.784494,0.013416
360,1383507660,207.59295,207.75000,207.59295,207.75000,0.100000,2,0,203.769407,205.688155,...,3.149727,23.696407,0.73623,0.73623,0.00000,0.329004,0.245671,57.250430,201.851973,0.013767
361,1383507720,207.75000,207.75000,207.75000,207.75000,0.100000,1,0,203.835202,205.755757,...,3.149727,23.694475,0.00000,0.00000,0.00000,0.325282,0.242892,57.250430,201.918675,0.012649
362,1383510600,207.75000,207.75000,207.75000,207.75000,0.314611,1,0,203.899909,205.821142,...,3.149727,23.696210,0.00000,0.00000,0.00000,0.321603,0.240145,57.250430,201.984609,0.009771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1737853,1590987240,9548.10000,9549.80000,9548.10000,9549.80000,0.599862,15,0,9546.714275,9555.564116,...,60.864865,26.509862,5.20000,5.20000,0.00000,2.058900,1.916728,51.788049,9537.146659,0.006402
1737854,1590987300,9546.20000,9546.20000,9546.20000,9546.20000,0.114211,2,0,9546.705775,9555.257096,...,64.756757,26.664156,-3.60000,0.00000,3.60000,2.036023,1.935431,51.266446,9537.247252,0.006402
1737855,1590987360,9547.40000,9547.40000,9544.00000,9545.20000,7.608574,15,0,9546.680886,9554.927355,...,65.837838,26.792956,-1.00000,0.00000,1.00000,2.013401,1.925037,51.121814,9537.335616,0.006402
1737856,1590987420,9543.90000,9544.00000,9543.90000,9543.90000,0.618188,5,0,9546.634921,9554.565802,...,67.243243,26.892546,-1.30000,0.00000,1.30000,1.991030,1.918092,50.932915,9537.408553,0.006475


In [12]:
#split the data and such. Split so that may use model predictions as features
X = appended_dataset.drop("peak_trough_wait", axis=1)
y = appended_dataset[["peak_trough_wait"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [13]:
#lets test out some classifying models
#Decision Tree
dec_tree = DecisionTreeClassifier()
dec_tree.fit(X_train.values, y_train.values)
dec_tree.score(X_test.values, y_test.values)


0.9356962972556807

In [14]:
#random forest
ran_forest = RandomForestClassifier(n_jobs = -1)
ran_forest.fit(X_train.values, y_train.values)
ran_forest.score(X_test.values, y_test.values)

<ipython-input-14-323335def364>:3: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



0.9806674437680784

In [15]:
#weights for rf
y_vals = y_train.values

buy_weight = len([x for x in y_vals if x== -1])
sell_weight = len([x for x in y_vals if x == 1])
zero_weight = len([x for x in y_vals if x != -1 and x != 1])
print(buy_weight, zero_weight, sell_weight)

30264 1329251 30264


In [16]:
#using rf model with weighted classes
class_weights = {-1: buy_weight, 0: zero_weight, 1: sell_weight}
weight_rf_forest = RandomForestClassifier(n_jobs=-1, class_weight=class_weights)
weight_rf_forest.fit(X_train, y_train)
weight_rf_forest.score(X_test, y_test)

<ipython-input-16-1a9723687244>:4: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



0.9739584682467729

In [48]:
boost = XGBClassifier(n_jobs= -1)
boost.fit(X_train.values, y_train.values)
boost.score(X_test.values, y_test.values)

/usr/local/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



0.9793291024478694

In [26]:
#weights for boost
boost_weights = []
for x in y_vals:
    if x == 0:
        boost_weights.append(zero_weight/(buy_weight + zero_weight + sell_weight))
    elif x == 1:
        boost_weights.append(sell_weight/(buy_weight + zero_weight + sell_weight))
    elif x == -1:
        boost_weights.append(buy_weight/(buy_weight + zero_weight + sell_weight))
    else:
        raise ValueError("this is the value in validation data")

In [27]:
boost = XGBClassifier(n_jobs= -1)
boost.fit(X_train.values, y_train.values, sample_weight=boost_weights)
boost.score(X_test.values, y_test.values)

/usr/local/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



0.9810675070874527

In [18]:
log_reg = LogisticRegression()
log_reg.fit(X_train.values, y_train.values)
log_reg.score(X_test.values, y_test.values)

/usr/local/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



0.9810675070874527

In [19]:
X_train

,time,open,high,low,close,volume,trades,ema_slow,ema_fast,macd,...,stosc_k,stosc_d,rsi_diff,rsi_u,rsi_d,rsi_smma_u,rsi_smma_d,rsi,smma,slope
358,1383503820,207.67198,207.67198,207.67198,207.67198,0.100000,1,203.646855,205.570957,1.924102,...,4.275846,23.660815,-0.00418,0.00000,0.00418,0.328056,0.243793,57.367632,201.724654,0.013416
359,1383505860,207.43988,207.43988,206.89000,207.01377,1.263692,5,203.702506,205.618262,1.915756,...,13.776266,23.698312,-0.65821,0.00000,0.65821,0.324344,0.248482,56.621691,201.784494,0.013416
360,1383507660,207.59295,207.75000,207.59295,207.75000,0.100000,2,203.769407,205.688155,1.918748,...,3.149727,23.696407,0.73623,0.73623,0.00000,0.329004,0.245671,57.250430,201.851973,0.013767
361,1383507720,207.75000,207.75000,207.75000,207.75000,0.100000,1,203.835202,205.755757,1.920555,...,3.149727,23.694475,0.00000,0.00000,0.00000,0.325282,0.242892,57.250430,201.918675,0.012649
362,1383510600,207.75000,207.75000,207.75000,207.75000,0.314611,1,203.899909,205.821142,1.921233,...,3.149727,23.696210,0.00000,0.00000,0.00000,0.321603,0.240145,57.250430,201.984609,0.009771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390408,1568168400,10081.90000,10081.90000,10075.10000,10075.20000,0.444556,9,10109.184922,10100.992946,-8.191976,...,95.321637,51.682985,-6.50000,0.00000,6.50000,1.748165,2.127896,45.101577,10108.996124,0.003531
1390409,1568168460,10075.20000,10075.20000,10075.20000,10075.20000,0.010462,1,10108.623188,10100.147276,-8.475912,...,95.252226,52.020408,0.00000,0.00000,0.00000,1.728740,2.104253,45.101577,10108.620612,0.002959
1390410,1568168580,10073.20000,10073.60000,10073.20000,10073.60000,0.052200,2,10108.044292,10099.276873,-8.767419,...,96.834817,52.355187,-1.60000,0.00000,1.60000,1.709532,2.098650,44.891028,10108.231494,0.003058
1390411,1568168640,10073.50000,10073.90000,10073.50000,10073.90000,0.015000,2,10107.479924,10098.444845,-9.035079,...,96.482412,52.689484,0.30000,0.30000,0.00000,1.693871,2.075332,44.939765,10107.850033,0.003058


In [30]:
#analasis of each method
rf_pred = ran_forest.predict(X_test.values)
rf_weight_pred = weight_rf_forest.predict(X_test.values)
dec_pred = dec_tree.predict(X_test.values)
boost_pred = boost.predict(X_test.values)
weights_boost_pred = weights_boost.predict(X_test,values)
log_reg_pred = log_reg.predict(X_test.values)

count_rf_pred = collections.Counter(rf_pred)
count_rf_weight_pred = collections.Counter(rf_weight_pred)
count_dec_pred = collections.Counter(dec_pred)
count_boost_pred = collections.Counter(boost_pred)
count_weights_boost_pred = collections.Counter(weights_boost_pred)
count_log_pred = collections.Counter(log_reg_pred)

print(count_rf_pred)
print(count_rf_weight_pred)
print(count_dec_pred)
print(count_boost_pred)
print(count_weights_boost_pred)
print(count_log_pred)

Counter({0: 347066, 1: 266, -1: 113})
Counter({0: 342221, 1: 2771, -1: 2453})
Counter({0: 328433, 1: 10635, -1: 8377})
Counter({0: 347445})
Counter({0: 347445})


In [64]:
#dec in RF with just predict not probability of each
RF_copy_X_train = X_train.copy()
RF_copy_X_test = X_test.copy()

RF_copy_X_train["dec_tree_pred"] = dec_tree.predict(X_train.values)
RF_copy_X_test["dec_tree_pred"] = dec_tree.predict(X_test.values)

rf_dec_mod = RandomForestClassifier(class_weight=class_weights, n_jobs=-1)
rf_dec_mod.fit(RF_copy_X_train.values, y_train.values)
rf_dec_mod.score(RF_copy_X_test.values, y_test.values)

<ipython-input-64-f0de0588b75b>:9: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



0.937017369655629

In [65]:
#dec in boost just predict not probability of each
boost_dec_mod = XGBClassifier(n_jobs=-1)
boost_dec_mod.fit(RF_copy_X_train.values, y_train.values)
boost_dec_mod.score(RF_copy_X_test.values, y_test.values)

/usr/local/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



0.937017369655629

In [49]:
#boost in RF
RF_boost_X_train = X_train.copy()
RF_boost_X_test = X_test.copy()

RF_boost_X_train["boost_pred"] = boost.predict(X_train.values)
RF_boost_X_test["boost_pred"] = boost.predict(X_test.values)

rf_boost_mod = RandomForestClassifier(class_weight=class_weights, n_jobs=-1)
rf_boost_mod.fit(RF_boost_X_train.values, y_train.values)
rf_boost_mod.score(RF_boost_X_test.values, y_test.values)

<ipython-input-49-323e58ba5d1f>:9: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



0.9735181107801235

In [50]:
#boost proba in rf
RF_boost_prob_X_train = X_train.copy()
RF_boost_prob_X_test = X_test.copy()

RF_boost_prob_train_pred = boost.predict_proba(X_train.values)
RF_boost_prob_test_pred = boost.predict_proba(X_test.values)

RF_boost_prob_X_train["neg_val"] = RF_boost_prob_train_pred[:,0]
RF_boost_prob_X_train["zero_val"] = RF_boost_prob_train_pred[:,1]
RF_boost_prob_X_train["pos_val"] = RF_boost_prob_train_pred[:,2]

RF_boost_prob_X_test["neg_val"] = RF_boost_prob_test_pred[:,0]
RF_boost_prob_X_test["zero_val"] = RF_boost_prob_test_pred[:,1]
RF_boost_prob_X_test["pos_val"] = RF_boost_prob_test_pred[:,2]

rf_boost_prob_mod = RandomForestClassifier(class_weight=class_weights, n_jobs=-1)
rf_boost_prob_mod.fit(RF_boost_prob_X_train.values, y_train.values)
rf_boost_prob_mod.score(RF_boost_prob_X_test.values, y_test.values)

<ipython-input-50-64795c7e4025>:17: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



0.9568679934953733

In [51]:
#RF in boost
boost_RF_X_train = X_train.copy()
boost_RF_X_test = X_test.copy()

boost_RF_X_train["rf_pred"] = weight_rf_forest.predict(X_train.values)
boost_RF_X_test["rf_pred"] = weight_rf_forest.predict(X_test.values)

boost_rf_mod = XGBClassifier(n_jobs=-1)
boost_rf_mod.fit(boost_RF_X_train.values, y_train.values)
boost_rf_mod.score(boost_RF_X_test.values, y_test.values)

/usr/local/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



0.9739584682467729

In [52]:
#rf proba in boost
boost_RF_prob_X_train = X_train.copy()
boost_RF_prob_X_test = X_test.copy()

boost_RF_prob_train_pred = boost.predict_proba(X_train.values)
boost_RF_prob_test_pred = boost.predict_proba(X_test.values)

boost_RF_prob_X_train["neg_val"] = boost_RF_prob_train_pred[:,0]
boost_RF_prob_X_train["zero_val"] = boost_RF_prob_train_pred[:,1]
boost_RF_prob_X_train["pos_val"] = boost_RF_prob_train_pred[:,2]

boost_RF_prob_X_test["neg_val"] = boost_RF_prob_test_pred[:,0]
boost_RF_prob_X_test["zero_val"] = boost_RF_prob_test_pred[:,1]
boost_RF_prob_X_test["pos_val"] = boost_RF_prob_test_pred[:,2]

boost_rf_prob_mod = XGBClassifier(n_jobs=-1)
boost_rf_prob_mod.fit(boost_RF_prob_X_train.values, y_train.values)
boost_rf_prob_mod.score(boost_RF_prob_X_test.values, y_test.values)

0.9703492639122739

In [54]:
#analysis of model in model
rf_dec_pred = rf_dec_mod.predict(RF_copy_X_test.values)
boost_dec_pred = boost_dec_mod.predict(boost_copy_X_test.values)
rf_boost_mod_pred = rf_boost_mod.predict(RF_boost_X_test.values)
rf_boost_prob_mod_pred = rf_boost_prob_mod.predict(RF_boost_prob_X_test.values)
boost_rf_mod_pred = boost_rf_mod.predict(boost_RF_X_test.values)
boost_rf_prob_mod_pred = boost_rf_prob_mod.predict(boost_RF_prob_X_test.values)

count_rf_dec = collections.Counter(rf_dec_pred)
count_boost_dec = collections.Counter(boost_dec_pred)
count_rf_boost_mod = collections.Counter(rf_boost_mod_pred)
count_rf_boost_prob_mod = collections.Counter(rf_boost_prob_mod_pred)
count_boost_rf_mod = collections.Counter(boost_rf_mod_pred)
count_boost_rf_prob_mod = collections.Counter(boost_rf_prob_mod_pred)

print(count_rf_dec)
print(count_boost_dec)
print(count_rf_boost_mod)
print(count_rf_boost_prob_mod)
print(count_boost_rf_mod)
print(count_boost_rf_prob_mod)


Counter({0: 342036, 1: 3180, -1: 2229})
Counter({0: 333983, 1: 9380, -1: 4082})
Counter({0: 342221, 1: 2771, -1: 2453})
Counter({0: 340883, 1: 5078, -1: 1484})


In [41]:
#dec in RF with just predict not probability of each
#TODO
rf_dec_train_X = X_train.copy()
rf_dec_test_X = X_test.copy()

X_train_dec_predict = dec_tree.predict_proba(X_train.values)
X_test_dec_predict = dec_tree.predict_proba(X_test.values)

rf_dec_train_X["neg_weight"] = X_train_dec_predict[:,0]
rf_dec_train_X["zero_weight"] = X_train_dec_predict[:,1]
rf_dec_train_X["pos_weight"] = X_train_dec_predict[:,2]

rf_dec_test_X["neg_weight"] = X_test_dec_predict[:,0]
rf_dec_test_X["zero_weight"] = X_test_dec_predict[:,1]
rf_dec_test_X["pos_weight"] = X_test_dec_predict[:,2]

rf_probs_mod = RandomForestClassifier(n_jobs=-1, class_weight=class_weights)
rf_probs_mod.fit(rf_dec_train_X.values, y_train.values)
rf_probs_mod.score(rf_dec_test_X.values, y_test.values)

<ipython-input-41-d6ef2a4f357d>:18: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



0.9356962972556807

In [42]:
#dec in RF with just predict not probability of each
boost_dec_train_X = X_train.copy()
boost_dec_test_X = X_test.copy()

boost_dec_train_X["neg_weight"] = X_train_dec_predict[:,0]
boost_dec_train_X["zero_weight"] = X_train_dec_predict[:,1]
boost_dec_train_X["pos_weight"] = X_train_dec_predict[:,2]

boost_dec_test_X["neg_weight"] = X_test_dec_predict[:,0]
boost_dec_test_X["zero_weight"] = X_test_dec_predict[:,1]
boost_dec_test_X["pos_weight"] = X_test_dec_predict[:,2]

boost_probs_mod = XGBClassifier(n_jobs= -1)
boost_probs_mod.fit(boost_dec_train_X.values, y_train.values)
boost_probs_mod.score(boost_dec_test_X.values, y_test.values)

/usr/local/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



0.9356962972556807

In [46]:
#analysis on the probability of the different models
rf_probs_pred = rf_probs_mod.predict(rf_dec_test_X.values)
boost_probs_pred = boost_probs_mod.predict(boost_dec_test_X.values)

count_rf_probs_pred = collections.Counter(rf_probs_pred)
count_boost_probs_pred = collections.Counter(boost_probs_pred)

print(count_rf_probs_pred)
print(count_boost_probs_pred)

Counter({0: 328433, 1: 10635, -1: 8377})
Counter({0: 328433, 1: 10635, -1: 8377})


In [21]:
#two mods in RF
rf_mult_X_train = X_train.copy()
rf_mult_X_test = X_test.copy()

rf_mult_X_train["dec_pred"] = dec_tree.predict(X_train.values)
rf_mult_X_train["boost_pred"] = boost.predict(X_train.values)
rf_mult_X_test["dec_pred"] = dec_tree.predict(X_test.values)
rf_mult_X_test["boost_pred"] = boost.predict(X_test.values)

rf_mult_mod = RandomForestClassifier(n_jobs=-1, class_weight=class_weights)
rf_mult_mod.fit(rf_mult_X_train.values, y_train.values)
rf_mult_mod.score(rf_mult_X_test.values, y_test.values)

<ipython-input-21-e8da793dde59>:11: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



0.9356962972556807

In [22]:
boost_mult_X_train = X_train.copy()
boost_mult_X_test = X_test.copy()

boost_mult_X_train["dec_pred"] = dec_tree.predict(X_train.values)
boost_mult_X_train["rf_pred"] = weight_rf_forest.predict(X_train.values)
boost_mult_X_test["dec_pred"] = dec_tree.predict(X_test.values)
boost_mult_X_test["rf_pred"] = weight_rf_forest.predict(X_test.values)

boost_mult_mod = XGBClassifier(n_jobs= -1)
boost_mult_mod.fit(boost_mult_X_train.values, y_train.values)
boost_mult_mod.score(boost_mult_X_test.values, y_test.values)

/usr/local/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



0.9356962972556807

In [47]:
#analysis of the two models in one
rf_mult_pred = rf_mult_mod.predict(rf_mult_X_test.values)
boost_mult_pred = boost_mult_mod.predict(boost_mult_X_test.values)

count_rf_mult_pred = collections.Counter(rf_mult_pred)
count_boost_mult_pred = collections.Counter(boost_mult_pred)

print(count_rf_mult_pred)
print(count_boost_mult_pred)

Counter({0: 328433, 1: 10635, -1: 8377})
Counter({0: 328433, 1: 10635, -1: 8377})
